In [1]:
import os

In [2]:
api_key = os.getenv("GEMINI_API_KEY")

In [3]:
from typing import Any, Dict

from google.adk.agents import Agent, LlmAgent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.adk.tools.tool_context import ToolContext
from google.genai import types

In [4]:
# Helper functions for running a session
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")

In [5]:
retry_config = types.HttpRetryOptions(
    attempts=3,
    exp_base=7, # Delay multiplier
    initial_delay=1, # initial delay before first retry (seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [6]:
APP_NAME = "Wellness Suite"
USER_ID = "Heekwon"
SESSION = "default"

MODEL_NAME = "gemini-2.5-flash-lite"

In [7]:
# Create Agents

# Fitness trainer agent
fitness_trainer_agent = Agent(
    name="FitnessTrainerAgent",
    model=Gemini(
        model_name="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are an expert fitness trainer agent. 
    Provide personalized workout plans based on user input. 
    Use the google_search tool to find exercises that address specific fitness goals.""",
    tools=[google_search],
    output_key="fitness_plan",
)

# Nutrition advisor agent
nutrition_advisor_agent = Agent(
    name="NutritionAdvisorAgent",
    model=Gemini(
        model_name="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a knowledgeable nutrition advisor agent. 
    Create customized meal plans based on dietary preferences and health goals. 
    Use the google_search tool to find healthy recipes and nutritional information.""",
    tools=[google_search],
    output_key="nutrition_plan",
)

# Coordinator root agent
root_agent = Agent(
    model=Gemini(model=MODEL_NAME, retry_options=retry_config),
    name="WellnessCoordinatorAgent",
    description="A holistic wellness coordinator agent",
    instruction="""You are the Wellness Coordinator Agent.
    Your goal is to assist users in improving their overall wellness.
    Based upon the user's input and previous conversation history,
    respond with advice to improve wellness in a concise and kind manner.
    Use the provided tools to provide fitness and nutrition advice if deemed necessary.
    """,
    tools=[
        AgentTool(fitness_trainer_agent),
        AgentTool(nutrition_advisor_agent)
    ],
)

In [8]:
# Set up Session Management
# InMemorySessionService stores conversations in RAM

session_service = InMemorySessionService()

In [9]:
# Create the Runner
runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)

print("Stateful agent initialized")
print(f" - Application: {APP_NAME}")
print(f" - User: {USER_ID}")
print(f" - Using: {session_service.__class__.__name__}")

Stateful agent initialized
 - Application: Wellness Suite
 - User: Heekwon
 - Using: InMemorySessionService


In [10]:
await run_session(
    runner,
    [
        "Hi, my name is Heekwon!"
    ],
    "stateful-agentic-session",
)


 ### Session: stateful-agentic-session

User > Hi, my name is Heekwon!
gemini-2.5-flash-lite >  Hi Heekwon, it's nice to meet you! How can I help you on your wellness journey today?


In [11]:
# Check persistence
await run_session(
    runner,
    [
        "I want to lose weight and gain muscle."
    ],
    "stateful-agentic-session",
)


 ### Session: stateful-agentic-session

User > I want to lose weight and gain muscle.
gemini-2.5-flash-lite >  That's a fantastic goal, Heekwon! To help you achieve that, we can work on both your nutrition and fitness.

For nutrition, would you like to focus on creating a calorie deficit for weight loss, or perhaps a balanced intake to support muscle gain?

For fitness, are you interested in strength training for muscle building, or a mix of cardio and strength? Once I have a better understanding of your preferences, I can provide more tailored advice!


In [12]:
await run_session(
    runner,
    [
        """I'd like to focus on a balanced intake to support muscle gain, and a mix of cardio and strength.
        I am 5 foot 6 and weigh 170 pounds.
        I am a male aged 30 and have previous weightlifting experience, but have fallen out of shape in the past few years."""
    ],
    "stateful-agentic-session",
)


 ### Session: stateful-agentic-session

User > I'd like to focus on a balanced intake to support muscle gain, and a mix of cardio and strength.
        I am 5 foot 6 and weigh 170 pounds.
        I am a male aged 30 and have previous weightlifting experience, but have fallen out of shape in the past few years.


gemini-2.5-flash-lite >  That's great information, Heekwon! It sounds like you have a good foundation to build upon.

Let's start with a balanced nutrition plan to support your muscle gain. We'll want to ensure you're getting enough protein, healthy fats, and complex carbohydrates.


gemini-2.5-flash-lite >  Heekwon, that's a fantastic plan! Combining a balanced nutrition approach with a mix of cardio and strength training is an excellent strategy for both losing weight and building muscle.

**Nutrition Focus:**

Your daily target is approximately **2944 - 3000 calories**, with a macronutrient breakdown of:
*   **Protein:** 170 g (essential for muscle repair and growth)
*   **Fats:** 83 g (important for hormone function and overall health)
*   **Carbohydrates:** 393 g (your primary energy source for workouts)

The sample meal plan provided offers a good mix of lean proteins, healthy fats, and complex carbohydrates throughout the day to keep you fueled and support muscle synthesis. Reme